필요한 라이브러리 가져오기

In [1]:
import requests
import pandas as pd
import bs4
import time
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


관광객 집계 데이터

In [4]:
with open('../APIkey.txt') as file:
    APIkey = file.read()

startdate = ['20220101', '20220201', '20220301', '20220401', '20220501', '20220601', '20220701', '20220801', '20220901', '20221001', '20221101', '20221201']
enddate = ['20220131', '20220228', '20220331', '20220430', '20220531', '20220630', '20220731', '20220831', '20220930', '20221031', '20221130', '20221231']

row_list = [] # 행값
value_list = [] # 데이터값

for s, e in tqdm(zip(startdate,enddate)):
    response = requests.get(f'https://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?numOfRows=13000&MobileOS=IOS&MobileApp=TestAPP&serviceKey={APIkey}&startYmd={s}&endYmd={e}')
    time.sleep(2)
    contents = response.text
    xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
    rows = xml_obj.findAll('item')
    for i in range(0, len(rows)): # xml 안의 데이터 수집
        columns = rows[i].find_all()
        for j in range(0,len(columns)): # 첫째 행 데이터 수집
            value_list.append(columns[j].text) # 컬럼의 각 데이터 값 저장
        row_list.append(value_list) # 각 행의 value값 전체 저장
        value_list=[] # 데이터 리스트 값 초기화


12it [00:46,  3.84s/it]


In [5]:
# xml값 DataFrame으로 만들기
df = pd.DataFrame(row_list)
df

,0,1,2,3,4,5,6,7
0,11,서울특별시,6,토요일,1,현지인(a),3698523.5,20220101
1,11,서울특별시,6,토요일,2,외지인(b),1171117.5,20220101
2,11,서울특별시,6,토요일,3,외국인(c),3806.8778575661786,20220101
3,26,부산광역시,6,토요일,1,현지인(a),1206936.5,20220101
4,26,부산광역시,6,토요일,2,외지인(b),445102.0,20220101
...,...,...,...,...,...,...,...,...
19817,48,경상남도,6,토요일,2,외지인(b),612486.0,20221231
19818,48,경상남도,6,토요일,3,외국인(c),556.3489851673804,20221231
19819,50,제주특별자치도,6,토요일,1,현지인(a),552450.5,20221231
19820,50,제주특별자치도,6,토요일,2,외지인(b),198383.5,20221231


In [6]:
# 컬럼명 변경
df.rename(columns={0:'areaCode', 1:'areaNm', 2:'daywkDivCd', 3:'daywkDivNm', 4:'touDivCd', 5:'touDivNm', 6:'touNum', 7:'baseYmd'}, inplace=True)
df

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
0,11,서울특별시,6,토요일,1,현지인(a),3698523.5,20220101
1,11,서울특별시,6,토요일,2,외지인(b),1171117.5,20220101
2,11,서울특별시,6,토요일,3,외국인(c),3806.8778575661786,20220101
3,26,부산광역시,6,토요일,1,현지인(a),1206936.5,20220101
4,26,부산광역시,6,토요일,2,외지인(b),445102.0,20220101
...,...,...,...,...,...,...,...,...
19817,48,경상남도,6,토요일,2,외지인(b),612486.0,20221231
19818,48,경상남도,6,토요일,3,외국인(c),556.3489851673804,20221231
19819,50,제주특별자치도,6,토요일,1,현지인(a),552450.5,20221231
19820,50,제주특별자치도,6,토요일,2,외지인(b),198383.5,20221231


In [7]:
# 현지인과 외국인 제외
df = df.loc[(df['touDivNm'] != '현지인(a)') & (df['touDivNm'] != '외국인(c)')]
df

,areaCode,areaNm,daywkDivCd,daywkDivNm,touDivCd,touDivNm,touNum,baseYmd
1,11,서울특별시,6,토요일,2,외지인(b),1171117.5,20220101
4,26,부산광역시,6,토요일,2,외지인(b),445102.0,20220101
7,27,대구광역시,6,토요일,2,외지인(b),264113.5,20220101
10,28,인천광역시,6,토요일,2,외지인(b),496692.0,20220101
13,29,광주광역시,6,토요일,2,외지인(b),161028.0,20220101
...,...,...,...,...,...,...,...,...
19808,45,전라북도,6,토요일,2,외지인(b),370013.0,20221231
19811,46,전라남도,6,토요일,2,외지인(b),475352.5,20221231
19814,47,경상북도,6,토요일,2,외지인(b),726484.5,20221231
19817,48,경상남도,6,토요일,2,외지인(b),612486.0,20221231


In [8]:
# 불필요한 컬럼삭제
df.drop(columns=['areaCode', 'daywkDivCd', 'touDivCd', 'touDivNm', 'touDivNm'], inplace=True)
df

,areaNm,daywkDivNm,touNum,baseYmd
1,서울특별시,토요일,1171117.5,20220101
4,부산광역시,토요일,445102.0,20220101
7,대구광역시,토요일,264113.5,20220101
10,인천광역시,토요일,496692.0,20220101
13,광주광역시,토요일,161028.0,20220101
...,...,...,...,...
19808,전라북도,토요일,370013.0,20221231
19811,전라남도,토요일,475352.5,20221231
19814,경상북도,토요일,726484.5,20221231
19817,경상남도,토요일,612486.0,20221231


In [9]:
# 인덱스값 재설정
df.set_index([pd.Index(range(0, 6613))], inplace=True)

In [10]:
# 결측값 확인
df.isnull().sum()

areaNm        0
daywkDivNm    0
touNum        0
baseYmd       0
dtype: int64

In [11]:
# 중복 확인
duplicate_index = df[df.duplicated()].index
print(duplicate_index)

Int64Index([3077, 3078, 3079, 3080, 3081, 3082, 3083, 3084, 3085, 3086,
            ...
            6552, 6553, 6554, 6555, 6556, 6557, 6558, 6559, 6560, 6561],
           dtype='int64', length=527)


In [12]:
# 중복값 제거
df.drop_duplicates(keep='first', ignore_index=True, inplace=True)

In [13]:
df

,areaNm,daywkDivNm,touNum,baseYmd
0,서울특별시,토요일,1171117.5,20220101
1,부산광역시,토요일,445102.0,20220101
2,대구광역시,토요일,264113.5,20220101
3,인천광역시,토요일,496692.0,20220101
4,광주광역시,토요일,161028.0,20220101
...,...,...,...,...
6081,전라북도,토요일,370013.0,20221231
6082,전라남도,토요일,475352.5,20221231
6083,경상북도,토요일,726484.5,20221231
6084,경상남도,토요일,612486.0,20221231


In [14]:
df['areaNm'].unique()

array(['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도'], dtype=object)

In [15]:
print('서울특별시 : ',  len(df.loc[df['areaNm'] == '서울특별시']))
print('부산광역시 : ',  len(df.loc[df['areaNm'] == '부산광역시']))
print('대구광역시 : ',  len(df.loc[df['areaNm'] == '대구광역시']))
print('인천광역시 : ',  len(df.loc[df['areaNm'] == '인천광역시']))
print('광주광역시 : ',  len(df.loc[df['areaNm'] == '광주광역시']))
print('울산광역시 : ',  len(df.loc[df['areaNm'] == '울산광역시']))
print('강원도 : ',  len(df.loc[df['areaNm'] == '강원도']))
print('전라북도 : ',  len(df.loc[df['areaNm'] == '전라북도']))
print('전라남도 : ',  len(df.loc[df['areaNm'] == '전라남도']))
print('충청북도 : ',  len(df.loc[df['areaNm'] == '충청북도']))
print('충청남도 : ',  len(df.loc[df['areaNm'] == '충청남도']))
print('경상북도 : ',  len(df.loc[df['areaNm'] == '경상북도']))
print('경상남도 : ',  len(df.loc[df['areaNm'] == '경상남도']))
print('제주특별자치도 : ',  len(df.loc[df['areaNm'] == '제주특별자치도']))
print('세종특별자치시 : ',  len(df.loc[df['areaNm'] == '세종특별자치시']))
print('경기도 : ',  len(df.loc[df['areaNm'] == '경기도']))
print('대전광역시 : ',  len(df.loc[df['areaNm'] == '대전광역시']))



서울특별시 :  358
부산광역시 :  358
대구광역시 :  358
인천광역시 :  358
광주광역시 :  358
울산광역시 :  358
강원도 :  358
전라북도 :  358
전라남도 :  358
충청북도 :  358
충청남도 :  358
경상북도 :  358
경상남도 :  358
제주특별자치도 :  358
세종특별자치시 :  358
경기도 :  358
대전광역시 :  358


In [16]:
df_seoul = df.loc[df['areaNm'] == '서울특별시']
df['baseYmd'].unique()

array(['20220101', '20220102', '20220103', '20220104', '20220105',
       '20220106', '20220107', '20220108', '20220109', '20220110',
       '20220111', '20220112', '20220113', '20220114', '20220115',
       '20220116', '20220117', '20220118', '20220119', '20220120',
       '20220121', '20220122', '20220123', '20220124', '20220125',
       '20220126', '20220127', '20220128', '20220129', '20220130',
       '20220131', '20220201', '20220202', '20220203', '20220204',
       '20220205', '20220206', '20220207', '20220208', '20220209',
       '20220210', '20220211', '20220212', '20220213', '20220214',
       '20220215', '20220216', '20220217', '20220218', '20220219',
       '20220220', '20220221', '20220222', '20220223', '20220224',
       '20220225', '20220226', '20220227', '20220228', '20220301',
       '20220302', '20220303', '20220304', '20220305', '20220306',
       '20220307', '20220308', '20220309', '20220310', '20220311',
       '20220312', '20220313', '20220314', '20220315', '202203

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6086 entries, 0 to 6085
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   areaNm      6086 non-null   object
 1   daywkDivNm  6086 non-null   object
 2   touNum      6086 non-null   object
 3   baseYmd     6086 non-null   object
dtypes: object(4)
memory usage: 190.3+ KB


In [18]:
df = df.astype({'touNum':'float'})

In [19]:
df.loc[df['areaNm'] == '서울특별시', 'touNum'].mean()

1522268.6480446928

In [20]:
# 결측값 총 7일 중앙값으로 대체 (없는날짜 20220816 ~ 20220820, 20220902 ~ 20220903)
null_day = ['20220816', '20220817', '20220818', '20220819', '20220820', '20220902', '20220903']
city = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도']
add_list = []

for j in null_day:
    for i in city:
        na_append = [i, df.loc[df['areaNm'] == i, 'touNum'].median(), j]
        add_list.append(na_append)

In [21]:
# 중앙값 기존 데이터 프레임과 합치기
df_add = pd.DataFrame(add_list)
df_add.rename(columns={0:'areaNm', 1:'touNum', 2:'baseYmd'}, inplace=True)
df_add = df_add.astype({'touNum' : 'float'})
df = pd.concat([df, df_add], axis=0)

In [22]:
df

,areaNm,daywkDivNm,touNum,baseYmd
0,서울특별시,토요일,1171117.50,20220101
1,부산광역시,토요일,445102.00,20220101
2,대구광역시,토요일,264113.50,20220101
3,인천광역시,토요일,496692.00,20220101
4,광주광역시,토요일,161028.00,20220101
...,...,...,...,...
114,전라북도,NaN,221235.00,20220903
115,전라남도,NaN,304470.25,20220903
116,경상북도,NaN,425582.50,20220903
117,경상남도,NaN,363550.25,20220903


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6205 entries, 0 to 118
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   areaNm      6205 non-null   object 
 1   daywkDivNm  6086 non-null   object 
 2   touNum      6205 non-null   float64
 3   baseYmd     6205 non-null   object 
dtypes: float64(1), object(3)
memory usage: 242.4+ KB


In [24]:
# 년월일 데이터 타입 및 형식 통일화를 위한 변경
df['baseYmd'] = pd.to_datetime(df['baseYmd'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
df['baseYmd'] = pd.to_datetime(df['baseYmd'])

In [25]:
# csv 저장
df.to_csv('./data/tour_data.csv', index=False, encoding='utf-8-sig')
df

,areaNm,daywkDivNm,touNum,baseYmd
0,서울특별시,토요일,1171117.50,2022-01-01
1,부산광역시,토요일,445102.00,2022-01-01
2,대구광역시,토요일,264113.50,2022-01-01
3,인천광역시,토요일,496692.00,2022-01-01
4,광주광역시,토요일,161028.00,2022-01-01
...,...,...,...,...
114,전라북도,NaN,221235.00,2022-09-03
115,전라남도,NaN,304470.25,2022-09-03
116,경상북도,NaN,425582.50,2022-09-03
117,경상남도,NaN,363550.25,2022-09-03


기상청 데이터

In [26]:
# 기상청 csv 형식 데이터 불러오기
weather = pd.read_csv('./data/OBS_ASOS_DD_20230317114852.csv', encoding='cp949')
weather.head()

,지점,지점명,일시,평균기온(°C),일강수량(mm),일 최심적설(cm)
0,101,춘천,2022-01-01,-8.2,NaN,NaN
1,101,춘천,2022-01-02,-2.7,0.0,NaN
2,101,춘천,2022-01-03,-3.8,0.0,NaN
3,101,춘천,2022-01-04,-3.4,NaN,NaN
4,101,춘천,2022-01-05,-4.7,NaN,NaN


In [27]:
# 지점명 변경을 위한 종류 확인
weather['지점명'].unique()

array(['춘천', '서울', '인천', '서산', '대전', '대구', '전주', '울산', '광주', '부산', '제주',
       '이천', '제천', '세종', '강진군', '봉화', '산청'], dtype=object)

In [28]:
# 각 지역에 맞게끔 변경
weather.replace('춘천', '강원도', inplace=True)
weather.replace('서울', '서울특별시', inplace=True)
weather.replace('인천', '인천광역시', inplace=True)
weather.replace('이천', '경기도', inplace=True)
weather.replace('대전', '대전광역시', inplace=True)
weather.replace('대구', '대구광역시', inplace=True)
weather.replace('전주', '전라북도', inplace=True)
weather.replace('울산', '울산광역시', inplace=True)
weather.replace('광주', '광주광역시', inplace=True)
weather.replace('부산', '부산광역시', inplace=True)
weather.replace('제주', '제주특별자치도', inplace=True)
weather.replace('제천', '충청북도', inplace=True)
weather.replace('서산', '충청남도', inplace=True)
weather.replace('세종', '세종특별자치시', inplace=True)
weather.replace('강진군', '전라남도', inplace=True)
weather.replace('산청', '경상남도', inplace=True)
weather.replace('봉화', '경상북도', inplace=True)

In [29]:
# 불필요한 컬럼 삭제
weather.drop(columns={'지점'}, inplace=True)

In [30]:
# 컬럼명 변경
weather.rename(columns={'지점명':'areaNm', '일시':'baseYmd', '평균기온(°C)':'avgTemp', '일강수량(mm)':'dayRf', '일 최심적설(cm)':'daySf'}, inplace=True)

In [31]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6203 entries, 0 to 6202
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   areaNm   6203 non-null   object 
 1   baseYmd  6203 non-null   object 
 2   avgTemp  6202 non-null   float64
 3   dayRf    2184 non-null   float64
 4   daySf    140 non-null    float64
dtypes: float64(3), object(2)
memory usage: 242.4+ KB


In [32]:
# 결측값 0.0 으로 통일
weather.fillna(0.0, inplace=True)

In [33]:
# 확인
weather

,areaNm,baseYmd,avgTemp,dayRf,daySf
0,강원도,2022-01-01,-8.2,0.0,0.0
1,강원도,2022-01-02,-2.7,0.0,0.0
2,강원도,2022-01-03,-3.8,0.0,0.0
3,강원도,2022-01-04,-3.4,0.0,0.0
4,강원도,2022-01-05,-4.7,0.0,0.0
...,...,...,...,...,...
6198,경상남도,2022-12-27,-1.9,0.0,0.0
6199,경상남도,2022-12-28,1.0,0.0,0.0
6200,경상남도,2022-12-29,0.0,0.0,0.0
6201,경상남도,2022-12-30,0.5,0.0,0.0


In [34]:
print('서울특별시 : ',  len(weather.loc[weather['areaNm'] == '서울특별시']))
print('부산광역시 : ',  len(weather.loc[weather['areaNm'] == '부산광역시']))
print('대구광역시 : ',  len(weather.loc[weather['areaNm'] == '대구광역시']))
print('인천광역시 : ',  len(weather.loc[weather['areaNm'] == '인천광역시']))
print('광주광역시 : ',  len(weather.loc[weather['areaNm'] == '광주광역시']))
print('울산광역시 : ',  len(weather.loc[weather['areaNm'] == '울산광역시']))
print('강원도 : ',  len(weather.loc[weather['areaNm'] == '강원도']))
print('전라북도 : ',  len(weather.loc[weather['areaNm'] == '전라북도']))
print('전라남도 : ',  len(weather.loc[weather['areaNm'] == '전라남도']))
print('충청북도 : ',  len(weather.loc[weather['areaNm'] == '충청북도']))
print('충청남도 : ',  len(weather.loc[weather['areaNm'] == '충청남도']))
print('경상북도 : ',  len(weather.loc[weather['areaNm'] == '경상북도']))
print('경상남도 : ',  len(weather.loc[weather['areaNm'] == '경상남도']))
print('제주특별자치도 : ',  len(weather.loc[weather['areaNm'] == '제주특별자치도']))
print('세종특별자치시 : ',  len(weather.loc[weather['areaNm'] == '세종특별자치시']))
print('경기도 : ',  len(weather.loc[weather['areaNm'] == '경기도']))
print('대전광역시 : ',  len(weather.loc[weather['areaNm'] == '대전광역시']))

서울특별시 :  365
부산광역시 :  365
대구광역시 :  365
인천광역시 :  365
광주광역시 :  365
울산광역시 :  365
강원도 :  365
전라북도 :  365
전라남도 :  365
충청북도 :  365
충청남도 :  365
경상북도 :  365
경상남도 :  365
제주특별자치도 :  365
세종특별자치시 :  363
경기도 :  365
대전광역시 :  365


In [35]:
weather_sejong = weather.loc[weather['areaNm'] == '세종특별자치시']
weather_sejong['baseYmd'].unique()

array(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04',
       '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08',
       '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12',
       '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16',
       '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20',
       '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24',
       '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28',
       '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01',
       '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05',
       '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09',
       '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13',
       '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17',
       '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21',
       '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25',
       '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01',
       '2022-03-02', '2022-03-03', '2022-03-04', '2022-

In [36]:
weather_sejong[270:310]

,areaNm,baseYmd,avgTemp,dayRf,daySf
5015,세종특별자치시,2022-09-28,18.9,0.0,0.0
5016,세종특별자치시,2022-10-01,19.1,0.0,0.0
5017,세종특별자치시,2022-10-02,20.6,1.6,0.0
5018,세종특별자치시,2022-10-03,23.1,24.0,0.0
5019,세종특별자치시,2022-10-04,19.6,45.5,0.0
5020,세종특별자치시,2022-10-05,15.6,0.0,0.0
5021,세종특별자치시,2022-10-06,15.7,1.5,0.0
5022,세종특별자치시,2022-10-07,14.8,0.0,0.0
5023,세종특별자치시,2022-10-08,15.7,0.0,0.0
5024,세종특별자치시,2022-10-09,12.8,22.3,0.0


In [37]:
# 결측값 직전일 근처값으로 대체 (없는날짜 20220929, 20220930)
       
add_list = [['세종특별자치시', '2022-09-29', 18.5, 0.0, 0.0], ['세종특별자치시', '2022-09-30', 18.5, 0.0, 0.0]]
weather_add = pd.DataFrame(add_list)
weather_add.rename(columns={0:'areaNm', 1:'baseYmd', 2:'avgTemp', 3:'dayRf', 4:'daySf'}, inplace=True)
weather = pd.concat([weather, weather_add], axis=0)
# 날짜 타입 변경
weather['baseYmd'] = pd.to_datetime(weather['baseYmd'])

In [38]:
# csv 저장
weather.to_csv('./data/weather.csv', index=False, encoding='utf-8')
weather

,areaNm,baseYmd,avgTemp,dayRf,daySf
0,강원도,2022-01-01,-8.2,0.0,0.0
1,강원도,2022-01-02,-2.7,0.0,0.0
2,강원도,2022-01-03,-3.8,0.0,0.0
3,강원도,2022-01-04,-3.4,0.0,0.0
4,강원도,2022-01-05,-4.7,0.0,0.0
...,...,...,...,...,...
6200,경상남도,2022-12-29,0.0,0.0,0.0
6201,경상남도,2022-12-30,0.5,0.0,0.0
6202,경상남도,2022-12-31,2.0,0.0,0.0
0,세종특별자치시,2022-09-29,18.5,0.0,0.0


지역축제 데이터

In [39]:
# csv 데이터 불러오기
festival = pd.read_csv("./data/전국문화축제표준데이터.csv", encoding='cp949')
festival.head(3)

,축제명,개최장소,축제시작일자,축제종료일자,축제내용,주관기관,주최기관,후원기관,전화번호,홈페이지주소,관련정보,소재지도로명주소,소재지지번주소,위도,경도,데이터기준일자,제공기관코드,제공기관명
0,강원대학교 산학협력단 K-Digital Training 제2기 교육생 모집,강원대학교 춘천캠퍼스,2022-10-04,2023-03-13,언리얼엔진 기반 메타버스 개발자 양성 과정 교육생 모집,고용노동부,강원대학교 산학협력단,NaN,033-250-8945,https://www.cccf.or.kr/Home/H40000/H40400/boar...,행사+축제,강원도 춘천시 강원대학길 1,강원도 춘천시 효자동 192-1,37.868024,127.748387,2023-01-12,4180000,강원도 춘천시
1,자연과 영혼을 담은 구자근展,춘천미술관 전관,2022-09-02,2022-09-07,자연과 영혼을 담은,구자근,구자근,NaN,033-241-1856,https://www.cccf.or.kr/Home/H40000/H40400/boar...,전시관람,강원도 춘천시 서부대성로 71,강원도 춘천시 옥천동 73-2,37.882374,127.731335,2023-01-12,4180000,강원도 춘천시
2,한동국 현관문,개나리미술관,2022-09-02,2022-09-18,"개나리미술관에서 원주출신,한동국의 첫 개인전을 개최",한동국,강원문화재단,NaN,070-8095-3899,https://www.cccf.or.kr/Home/H40000/H40400/boar...,전시관람,강원도 춘천시 동내면 거두택지길44번길 7-19,강원도 춘천시 동내면 거두리 1123-6,37.854547,127.758295,2023-01-12,4180000,강원도 춘천시


In [40]:
# 가독성을 위해 불필요한 컬럼 삭제
festival.drop(columns=['개최장소', '축제내용', '주관기관', '주최기관', '후원기관', '전화번호', '홈페이지주소', '소재지지번주소', '위도', '경도', '데이터기준일자', '제공기관코드', '제공기관명'], inplace=True)
festival

,축제명,축제시작일자,축제종료일자,관련정보,소재지도로명주소
0,강원대학교 산학협력단 K-Digital Training 제2기 교육생 모집,2022-10-04,2023-03-13,행사+축제,강원도 춘천시 강원대학길 1
1,자연과 영혼을 담은 구자근展,2022-09-02,2022-09-07,전시관람,강원도 춘천시 서부대성로 71
2,한동국 현관문,2022-09-02,2022-09-18,전시관람,강원도 춘천시 동내면 거두택지길44번길 7-19
3,2022 춘천학 학술대회 댐과 춘천의 미래,2022-08-26,2022-08-26,행사+축제,강원도 춘천시 강원대학길 1
4,할말하않 황효창展,2022-08-16,2022-08-28,전시관람,강원도 춘천시 동내면 거두택지길44번길 7-19
...,...,...,...,...,...
1360,여주오곡나루축제,2022-10-21,2022-10-23,신륵사+명성황후생가+영릉+황학산수목원,경기도 여주시 신륵사길 7
1361,김해분청도자기축제,2022-10-21,2022-10-30,김해분청도자박물관+클레이아크미술관,경상남도 김해시 진례면 진례로 275-35
1362,청송사과축제,2022-11-02,2022-11-06,NaN,경상북도 청송군 청송읍 금월로 269
1363,영덕대게축제,2022-12-27,2022-12-27,해파랑공원,경상북도 영덕군 강구면 영덕대게로 136


In [41]:
# 축제만 추출
festival = festival[festival['축제명'].str.contains('축제', na =False)]

In [42]:
# 개최취소된 축제 삭제
festival = festival.loc[festival['관련정보'] != '개최취소']

In [43]:
# 개최일 타입 변경
festival['축제시작일자'] = pd.to_datetime(festival['축제시작일자'])
festival['축제종료일자'] = pd.to_datetime(festival['축제종료일자'])

In [44]:
# 2022년 축제만 추출
festival = festival[festival['축제시작일자'].dt.year == 2022]
festival = festival[festival['축제종료일자'].dt.year == 2022]
festival

,축제명,축제시작일자,축제종료일자,관련정보,소재지도로명주소
18,불갑산상사화축제,2022-09-16,2022-09-25,NaN,전라남도 영광군 불갑면 불갑사로 450
21,청송사과축제,2022-11-09,2022-11-13,NaN,NaN
23,필봉마을굿축제,2022-08-12,2022-08-15,NaN,전라북도 임실군 강진면 강운로 272
24,임실N치즈축제,2022-10-07,2022-10-10,NaN,전라북도 임실군 성수면 도인2길 50
25,산타축제,2022-12-23,2022-12-25,NaN,전라북도 임실군 성수면 도인2길 50
...,...,...,...,...,...
1358,광주세계김치축제,2022-10-20,2022-10-23,NaN,광주광역시 남구 김치로 60
1360,여주오곡나루축제,2022-10-21,2022-10-23,신륵사+명성황후생가+영릉+황학산수목원,경기도 여주시 신륵사길 7
1361,김해분청도자기축제,2022-10-21,2022-10-30,김해분청도자박물관+클레이아크미술관,경상남도 김해시 진례면 진례로 275-35
1362,청송사과축제,2022-11-02,2022-11-06,NaN,경상북도 청송군 청송읍 금월로 269


In [45]:
# 축제기간 일수 확인을 위한 컬럼 생성 및 값 추가
festival['Days'] = festival['축제종료일자'] - festival['축제시작일자']


In [46]:
# 데이터프레임 연결을 위한 주소 분리 및 광역시명 컬럼 생성
festival['areaNm'] = festival['소재지도로명주소'].str.split(' ',n=1,expand=True)[0]

In [47]:
# 불필요해진 컬럼 삭제
festival.drop(columns=['관련정보', '소재지도로명주소'], inplace=True)
festival

,축제명,축제시작일자,축제종료일자,Days,areaNm
18,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
21,청송사과축제,2022-11-09,2022-11-13,4 days,NaN
23,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
24,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도
25,산타축제,2022-12-23,2022-12-25,2 days,전라북도
...,...,...,...,...,...
1358,광주세계김치축제,2022-10-20,2022-10-23,3 days,광주광역시
1360,여주오곡나루축제,2022-10-21,2022-10-23,2 days,경기도
1361,김해분청도자기축제,2022-10-21,2022-10-30,9 days,경상남도
1362,청송사과축제,2022-11-02,2022-11-06,4 days,경상북도


In [48]:
# 광역시 종류 확인
festival['areaNm'].unique()

array(['전라남도', nan, '전라북도', '울산광역시', '광주광역시', '경기도', '인천광역시', '강원도',
       '서울특별시', '충청북도', '충청남도', '제주특별자치도', '경상남도', '부산광역시', '경상북도',
       '대전광역시', '세종특별자치시', '세종특별자치시조치원읍문예회관길22', '미정', '울릉군', '대구광역시'],
      dtype=object)

In [49]:
# 오류값 제거
festival = festival.loc[(festival['areaNm'] != 'nan') & 
                        (festival['areaNm'] != '세종특별자치시조치원읍문예회관길22') & 
                        (festival['areaNm'] != '울릉군') & 
                        (festival['areaNm'] != '미정')]

In [50]:
# null 값 제거
festival.dropna(axis=0, inplace=True)

In [51]:
# 축제기간이 한달 밑인 축제만 남기기
festival = festival.loc[festival['Days'] <= '30 Days']
festival

,축제명,축제시작일자,축제종료일자,Days,areaNm
18,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
23,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
24,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도
25,산타축제,2022-12-23,2022-12-25,2 days,전라북도
29,울산쇠부리축제,2022-05-13,2022-05-15,2 days,울산광역시
...,...,...,...,...,...
1358,광주세계김치축제,2022-10-20,2022-10-23,3 days,광주광역시
1360,여주오곡나루축제,2022-10-21,2022-10-23,2 days,경기도
1361,김해분청도자기축제,2022-10-21,2022-10-30,9 days,경상남도
1362,청송사과축제,2022-11-02,2022-11-06,4 days,경상북도


In [52]:
# 인덱스 번호 맞추기
festival.set_index([pd.Index(range(0, 317))], inplace=True)

In [53]:
# 컬럼명 변경
festival.rename(columns={'축제명':'festivalNm', '축제시작일자':'fStart', '축제종료일자':'fClose'}, inplace=True)

In [54]:
festival.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   festivalNm  317 non-null    object         
 1   fStart      317 non-null    datetime64[ns] 
 2   fClose      317 non-null    datetime64[ns] 
 3   Days        317 non-null    timedelta64[ns]
 4   areaNm      317 non-null    object         
dtypes: datetime64[ns](2), object(2), timedelta64[ns](1)
memory usage: 12.5+ KB


In [55]:
# 확인
festival

,festivalNm,fStart,fClose,Days,areaNm
0,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
1,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
2,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도
3,산타축제,2022-12-23,2022-12-25,2 days,전라북도
4,울산쇠부리축제,2022-05-13,2022-05-15,2 days,울산광역시
...,...,...,...,...,...
312,광주세계김치축제,2022-10-20,2022-10-23,3 days,광주광역시
313,여주오곡나루축제,2022-10-21,2022-10-23,2 days,경기도
314,김해분청도자기축제,2022-10-21,2022-10-30,9 days,경상남도
315,청송사과축제,2022-11-02,2022-11-06,4 days,경상북도


In [56]:
# csv 저장
festival.to_csv("./data/festival.csv", index=False, encoding='utf-8')

22년도 공휴일 데이터

In [57]:
with open('c:/Kcoding/Mini_Project/APIkey2.txt') as file:
    APIkey2 = file.read()

url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
params ={'serviceKey' : APIkey2, 'solYear' : '2022', 'numOfRows' : '50'}
row_list = []
value_list = []

response = requests.get(url, params=params)
time.sleep(2)
contents = response.text
xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
rows = xml_obj.findAll('item')
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    for j in range(0,len(columns)):
        value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]
holiday = pd.DataFrame(row_list)

100%|██████████| 19/19 [00:00<?, ?it/s]


In [58]:
holiday

,0,1,2,3,4
0,01,1월1일,Y,20220101,1
1,01,설날,Y,20220131,1
2,01,설날,Y,20220201,1
3,01,설날,Y,20220202,1
4,01,삼일절,Y,20220301,1
5,01,대통령선거일,Y,20220309,1
6,01,어린이날,Y,20220505,1
7,01,부처님오신날,Y,20220508,1
8,01,전국동시지방선거,Y,20220601,1
9,01,현충일,Y,20220606,1


In [59]:
holiday.drop(columns=[0, 2, 4], inplace=True)
holiday.rename(columns={1 : 'holNm', 3 : 'baseYmd'}, inplace=True)
holiday['baseYmd'] = pd.to_datetime(holiday['baseYmd'], format='%Y%m%d').dt.strftime('%Y-%m-%d')
holiday['baseYmd'] = pd.to_datetime(holiday['baseYmd'])
holiday

,holNm,baseYmd
0,1월1일,2022-01-01
1,설날,2022-01-31
2,설날,2022-02-01
3,설날,2022-02-02
4,삼일절,2022-03-01
5,대통령선거일,2022-03-09
6,어린이날,2022-05-05
7,부처님오신날,2022-05-08
8,전국동시지방선거,2022-06-01
9,현충일,2022-06-06


In [60]:
holiday.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   holNm    19 non-null     object        
 1   baseYmd  19 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 432.0+ bytes


In [61]:
holiday.to_csv('./data/holiday.csv', encoding='utf-8', index=False)

코로나 확진자 데이터

In [66]:
import datetime

start_date = datetime.date(2022, 1, 1)
end_date = datetime.date(2022, 12, 31)
delta = datetime.timedelta(days=1)

date_list = []
while start_date <= end_date:
    date_list.append(start_date.strftime("%Y-%m-%d"))
    start_date += delta
    
print(date_list)


['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23', '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27', '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31', '2022-02-01', '2022-02-02', '2022-02-03', '2022-02-04', '2022-02-05', '2022-02-06', '2022-02-07', '2022-02-08', '2022-02-09', '2022-02-10', '2022-02-11', '2022-02-12', '2022-02-13', '2022-02-14', '2022-02-15', '2022-02-16', '2022-02-17', '2022-02-18', '2022-02-19', '2022-02-20', '2022-02-21', '2022-02-22', '2022-02-23', '2022-02-24', '2022-02-25', '2022-02-26', '2022-02-27', '2022-02-28', '2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04', '2022-03-05', '2022-03-06', '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11', '2022-03-12', '2022

In [83]:
with open('c:/Kcoding/Mini_Project/APIkey2.txt') as file:
    APIkey2 = file.read()

In [102]:
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'

row_list = []
value_list = []

for date in tqdm(date_list):
    params = {'serviceKey' : APIkey2, 'pageNo' : '1', 'numOfRows' : '500', 'apiType' : 'xml', 'std_day' : date }
    response = requests.get(url, params=params)
    requestData = requests.get('REST API URL')
    contents = response.text
    xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
    rows = xml_obj.findAll('item')
    for i in range(0, len(rows)):
        columns = rows[i].find_all()
        for j in range(0,len(columns)):
            value_list.append(columns[j].text)
        row_list.append(value_list)
        value_list=[]

100%|██████████| 365/365 [20:31<00:00,  3.37s/it]


In [109]:
covid19 = pd.DataFrame(row_list)

In [110]:
covid19

,0,1,2,3,4,5,6,7,8,9,10,11
0,38,6126,전남,全南,Jeollanam-do,48,5404,0,47,1,331,2022-01-01
1,266,36962,인천,仁川,Incheon,241,30703,0,239,2,1256,2022-01-01
2,5781,645226,합계,合计,Total,3024,537043,0,2889,135,1249,2022-01-04
3,16,6926,검역,隔離區,Lazaretto,49,6486,0,0,49,-,2022-01-05
4,63,7101,울산,蔚山,Ulsan,48,6570,0,47,1,633,2022-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...
355,1938,1304161,경북,庆北,Gyeongsangbuk-do,2832,0,0,2830,2,49652,2022-12-23
356,874,837401,대전,大田,Daejeon,1704,0,0,1701,3,57662,2022-12-25
357,967,910483,충북,忠北,Chungcheongbuk-do,2007,0,0,2006,1,56997,2022-12-30
358,50,230222,세종,世宗,Sejong,465,0,0,464,1,61905,2022-12-31


In [ ]:
# 광역시명 변경 및 합계, 검역 값 행 제거
covid19 = covid19.loc[(covid19[2] != '검역') & (covid19[2] != '합계')]
covid19.replace('경기', '경기도', inplace=True)
covid19.replace('강원', '강원도', inplace=True)
covid19.replace('경북', '경상북도', inplace=True)
covid19.replace('대전', '대전광역시', inplace=True)
covid19.replace('울산', '울산광역시', inplace=True)
covid19.replace('대구', '대구광역시', inplace=True)
covid19.replace('제주', '제주특별자치도', inplace=True)
covid19.replace('충북', '충청북도', inplace=True)
covid19.replace('세종', '세종특별자치시', inplace=True)
covid19.replace('광주', '광주광역시', inplace=True)
covid19.replace('서울', '서울특별시', inplace=True)
covid19.replace('전북', '전라북도', inplace=True)
covid19.replace('부산', '부산광역시', inplace=True)
covid19.replace('경남', '경상남도', inplace=True)
covid19.replace('충남', '충청남도', inplace=True)
covid19.replace('인천', '인천광역시', inplace=True)
covid19.replace('전남', '전라남도', inplace=True)

In [ ]:
# 불필요한 컬럼 삭제 및 컬럼명 변경
covid19.drop(columns=[0, 3, 4, 7, 10], inplace=True)
covid19.rename(columns={1:'AccumulateCovid19', 2:'areaNm', 5:'increase', 6:'AccumulateReleased', 8:'localIncrease', 9:'CovidForeigner', 11:'baseYmd'}, inplace=True)
covid19

In [95]:
covid19 = pd.read_csv('c:/Kcoding/Mini_Project/backup/covid19.csv')
covid19 = covid19.loc[covid19['baseYmd'].str.contains('2022')]
covid19

,AccumulateCovid19,areaNm,increase,AccumulateReleased,localIncrease,CovidForeigner,baseYmd
0,6923,울산광역시,50,6371,50,0,2022-01-01
1,2158,세종특별자치시,16,1957,16,0,2022-01-01
2,11602,충청북도,104,10540,103,1,2022-01-01
3,4661,제주특별자치도,23,4446,22,1,2022-01-01
4,9653,전라북도,85,8249,84,1,2022-01-01
...,...,...,...,...,...,...,...
6182,5663251,서울특별시,11601,0,11597,4,2022-12-31
6183,1686864,인천광역시,3976,0,3975,1,2022-12-31
6184,365960,제주특별자치도,691,0,691,0,2022-12-31
6185,849060,광주광역시,1615,0,1612,3,2022-12-31


In [96]:
covid19['baseYmd'] = pd.to_datetime(covid19['baseYmd'])

In [97]:
print('서울특별시 : ',  len(covid19.loc[covid19['areaNm'] == '서울특별시']))
print('부산광역시 : ',  len(covid19.loc[covid19['areaNm'] == '부산광역시']))
print('대구광역시 : ',  len(covid19.loc[covid19['areaNm'] == '대구광역시']))
print('인천광역시 : ',  len(covid19.loc[covid19['areaNm'] == '인천광역시']))
print('광주광역시 : ',  len(covid19.loc[covid19['areaNm'] == '광주광역시']))
print('울산광역시 : ',  len(covid19.loc[covid19['areaNm'] == '울산광역시']))
print('강원도 : ',  len(covid19.loc[covid19['areaNm'] == '강원도']))
print('전라북도 : ',  len(covid19.loc[covid19['areaNm'] == '전라북도']))
print('전라남도 : ',  len(covid19.loc[covid19['areaNm'] == '전라남도']))
print('충청북도 : ',  len(covid19.loc[covid19['areaNm'] == '충청북도']))
print('충청남도 : ',  len(covid19.loc[covid19['areaNm'] == '충청남도']))
print('경상북도 : ',  len(covid19.loc[covid19['areaNm'] == '경상북도']))
print('경상남도 : ',  len(covid19.loc[covid19['areaNm'] == '경상남도']))
print('제주특별자치도 : ',  len(covid19.loc[covid19['areaNm'] == '제주특별자치도']))
print('세종특별자치시 : ',  len(covid19.loc[covid19['areaNm'] == '세종특별자치시']))
print('경기도 : ',  len(covid19.loc[covid19['areaNm'] == '경기도']))
print('대전광역시 : ',  len(covid19.loc[covid19['areaNm'] == '대전광역시']))

서울특별시 :  319
부산광역시 :  340
대구광역시 :  329
인천광역시 :  336
광주광역시 :  319
울산광역시 :  344
강원도 :  331
전라북도 :  324
전라남도 :  327
충청북도 :  337
충청남도 :  321
경상북도 :  318
경상남도 :  331
제주특별자치도 :  329
세종특별자치시 :  328
경기도 :  330
대전광역시 :  336


In [98]:
date = pd.DataFrame(date_list)
date.rename(columns={0:'baseYmd'}, inplace=True)
date = pd.to_datetime(date['baseYmd'])

In [99]:
areaNm = ['울산광역시', '세종특별자치시', '충청북도', '제주특별자치도', '전라북도', '경기도', '부산광역시',
       '서울특별시', '대전광역시', '경상남도', '경상북도', '광주광역시', '강원도', '충청남도', '인천광역시',
       '전라남도', '대구광역시']

empted_df = pd.DataFrame()
covid19_full = pd.DataFrame()

for area in areaNm:
    empted_df = covid19.loc[covid19['areaNm'] == area]
    covid19_na = pd.merge(date, empted_df, left_on=['baseYmd'], right_on=['baseYmd'], how='outer')
    empted_df = pd.DataFrame()
    covid19_na.drop_duplicates(subset=['baseYmd'], ignore_index=True, inplace=True)
    
    # 각 컬럼마다 직전일 값으로 대체
    for col in covid19_na.columns:
        covid19_na[col] = covid19_na[col].fillna(method='ffill')
    
    covid19_full = pd.concat([covid19_full, covid19_na], axis=0)

In [100]:
covid19_full.isnull().sum()

baseYmd               0
AccumulateCovid19     0
areaNm                0
increase              0
AccumulateReleased    0
localIncrease         0
CovidForeigner        0
dtype: int64

In [101]:
covid19_full.sort_values(['baseYmd'])

,baseYmd,AccumulateCovid19,areaNm,increase,AccumulateReleased,localIncrease,CovidForeigner
0,2022-01-01,6923.0,울산광역시,50.0,6371.0,50.0,0.0
0,2022-01-01,8118.0,광주광역시,92.0,7349.0,91.0,1.0
0,2022-01-01,226698.0,서울특별시,1464.0,177348.0,1429.0,35.0
0,2022-01-01,15431.0,경상북도,116.0,13456.0,114.0,2.0
0,2022-01-01,2158.0,세종특별자치시,16.0,1957.0,16.0,0.0
...,...,...,...,...,...,...,...
364,2022-12-31,1686864.0,인천광역시,3976.0,0.0,3975.0,1.0
364,2022-12-31,947524.0,전라남도,2117.0,0.0,2116.0,1.0
364,2022-12-31,618200.0,울산광역시,1604.0,0.0,1603.0,1.0
364,2022-12-31,1716966.0,부산광역시,4502.0,0.0,4501.0,1.0


In [102]:
print('서울특별시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '서울특별시']))
print('부산광역시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '부산광역시']))
print('대구광역시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '대구광역시']))
print('인천광역시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '인천광역시']))
print('광주광역시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '광주광역시']))
print('울산광역시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '울산광역시']))
print('강원도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '강원도']))
print('전라북도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '전라북도']))
print('전라남도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '전라남도']))
print('충청북도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '충청북도']))
print('충청남도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '충청남도']))
print('경상북도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '경상북도']))
print('경상남도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '경상남도']))
print('제주특별자치도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '제주특별자치도']))
print('세종특별자치시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '세종특별자치시']))
print('경기도 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '경기도']))
print('대전광역시 : ',  len(covid19_full.loc[covid19_full['areaNm'] == '대전광역시']))

서울특별시 :  365
부산광역시 :  365
대구광역시 :  365
인천광역시 :  365
광주광역시 :  365
울산광역시 :  365
강원도 :  365
전라북도 :  365
전라남도 :  365
충청북도 :  365
충청남도 :  365
경상북도 :  365
경상남도 :  365
제주특별자치도 :  365
세종특별자치시 :  365
경기도 :  365
대전광역시 :  365


In [72]:
covid19_full['baseYmd'] = pd.to_datetime(covid19['baseYmd'])

In [103]:
covid19_full.to_csv('./data/covid19_final.csv', encoding='utf-8', index=False)
covid19_full

,baseYmd,AccumulateCovid19,areaNm,increase,AccumulateReleased,localIncrease,CovidForeigner
0,2022-01-01,6923.0,울산광역시,50.0,6371.0,50.0,0.0
1,2022-01-02,6981.0,울산광역시,58.0,6411.0,58.0,0.0
2,2022-01-03,7019.0,울산광역시,38.0,6453.0,38.0,0.0
3,2022-01-04,7053.0,울산광역시,34.0,6517.0,33.0,1.0
4,2022-01-05,7101.0,울산광역시,48.0,6570.0,47.0,1.0
...,...,...,...,...,...,...,...
360,2022-12-27,1257368.0,대구광역시,3464.0,0.0,3458.0,6.0
361,2022-12-28,1260890.0,대구광역시,3522.0,0.0,3510.0,12.0
362,2022-12-29,1263538.0,대구광역시,2648.0,0.0,2646.0,2.0
363,2022-12-30,1266241.0,대구광역시,2703.0,0.0,2700.0,3.0


미세먼지 데이터

In [75]:
with open('c:/Kcoding/Mini_Project/APIkey2.txt') as file:
    APIkey2 = file.read()

row_list = []
value_list = []
url = 'http://apis.data.go.kr/B552584/UlfptcaAlarmInqireSvc/getUlfptcaAlarmInfo'
params = {'serviceKey' : APIkey2, 'returnType' : 'xml', 'numOfRows' : '12000', 'pageNo' : '1', 'year' : '2022'}

response = requests.get(url, params=params)
time.sleep(2)
contents = response.text
xml_obj = bs4.BeautifulSoup(contents,'lxml-xml')
rows = xml_obj.findAll('item')
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    for j in range(0,len(columns)):
        value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]

In [76]:
pm = pd.DataFrame(row_list)

In [77]:
pm

,0,1,2,3,4,5,6,7,8,9,10,11
0,29,373,충북,2022-12-27,75,14:00,2022-12-28,2022-12-27,북부권역,13:00,주의보,PM25
1,64,370,제주,2022-12-14,213,23:00,2022-12-14,2022-12-13,제주권역,01:00,주의보,PM10
2,136,308,경기,2022-12-13,311,06:00,2022-12-13,2022-12-13,중부권,16:00,경보,PM10
3,99,364,강원,2022-12-13,145,18:00,2022-12-13,2022-12-13,영서북부,19:00,주의보,PM10
4,76,357,인천,2022-12-13,105,17:00,2022-12-13,2022-12-13,서부권역,18:00,주의보,PM10
...,...,...,...,...,...,...,...,...,...,...,...,...
278,33,5,인천,2022-01-08,86,23:00,2022-01-11,2022-01-08,강화권역,02:00,주의보,PM25
279,27,11,인천,2022-01-08,83,00:00,2022-01-10,2022-01-09,동남부권역,20:00,주의보,PM25
280,23,1,서울,2022-01-08,82,22:00,2022-01-10,2022-01-08,서울권역,21:00,주의보,PM25
281,30,2,경기,2022-01-08,81,22:00,2022-01-11,2022-01-08,북부권,03:00,주의보,PM25


In [78]:
# 불필요한 컬럼 삭제 및 컬럼명 변경
pm.drop(columns=[0, 1, 5, 6, 7, 8, 9],inplace=True)
pm.rename(columns={2 : 'areaNm', 3 : 'baseYmd', 4 : 'CAI', 10 : 'advisory', 11 : 'pmcategory'}, inplace=True)

In [79]:
pm['areaNm'].unique()

array(['충북', '제주', '경기', '강원', '인천', '전남', '경북', '경남', '서울', '광주', '대전',
       '전북', '충남', '부산', '대구', '울산', '세종'], dtype=object)

In [80]:
pm.replace('경기', '경기도', inplace=True)
pm.replace('강원', '강원도', inplace=True)
pm.replace('경북', '경상북도', inplace=True)
pm.replace('대전', '대전광역시', inplace=True)
pm.replace('울산', '울산광역시', inplace=True)
pm.replace('대구', '대구광역시', inplace=True)
pm.replace('제주', '제주특별자치도', inplace=True)
pm.replace('충북', '충청북도', inplace=True)
pm.replace('세종', '세종특별자치시', inplace=True)
pm.replace('광주', '광주광역시', inplace=True)
pm.replace('서울', '서울특별시', inplace=True)
pm.replace('전북', '전라북도', inplace=True)
pm.replace('부산', '부산광역시', inplace=True)
pm.replace('경남', '경상남도', inplace=True)
pm.replace('충남', '충청남도', inplace=True)
pm.replace('인천', '인천광역시', inplace=True)
pm.replace('전남', '전라남도', inplace=True)

In [81]:
# 년월일 타입 변경
pm['baseYmd'] = pd.to_datetime(pm['baseYmd'])

In [82]:
# 중복값 확인
pm.duplicated().sum()

18

In [83]:
# 중복값 제거
pm.drop_duplicates(ignore_index=True, inplace=True)

In [84]:
# 확인
pm

,areaNm,baseYmd,CAI,advisory,pmcategory
0,충청북도,2022-12-27,75,주의보,PM25
1,제주특별자치도,2022-12-14,213,주의보,PM10
2,경기도,2022-12-13,311,경보,PM10
3,강원도,2022-12-13,145,주의보,PM10
4,인천광역시,2022-12-13,105,주의보,PM10
...,...,...,...,...,...
260,인천광역시,2022-01-08,86,주의보,PM25
261,인천광역시,2022-01-08,83,주의보,PM25
262,서울특별시,2022-01-08,82,주의보,PM25
263,경기도,2022-01-08,81,주의보,PM25


In [85]:
pm.to_csv('./data/pm.csv', index=False, encoding='utf-8')

데이터 합치기

In [2]:
# 관광객수 집계 데이터
df = pd.read_csv('./data/tour_data.csv', parse_dates=['baseYmd'])
df.head(3)

,areaNm,daywkDivNm,touNum,baseYmd
0,서울특별시,토요일,1171117.5,2022-01-01
1,부산광역시,토요일,445102.0,2022-01-01
2,대구광역시,토요일,264113.5,2022-01-01


In [3]:
# 날씨 데이터
weather = pd.read_csv('./data/weather.csv', parse_dates=['baseYmd'])
weather.head(3)

,areaNm,baseYmd,avgTemp,dayRf,daySf
0,강원도,2022-01-01,-8.2,0.0,0.0
1,강원도,2022-01-02,-2.7,0.0,0.0
2,강원도,2022-01-03,-3.8,0.0,0.0


In [4]:
# 지역축제 데이터
festival = pd.read_csv('./data/festival.csv', parse_dates=['fClose', 'fStart'])
festival.head(3)

,festivalNm,fStart,fClose,Days,areaNm
0,불갑산상사화축제,2022-09-16,2022-09-25,9 days,전라남도
1,필봉마을굿축제,2022-08-12,2022-08-15,3 days,전라북도
2,임실N치즈축제,2022-10-07,2022-10-10,3 days,전라북도


In [5]:
# 코로나 데이터
covid19 = pd.read_csv('./data/covid19_final.csv', parse_dates=['baseYmd'])
covid19.sort_values(['baseYmd'])

,baseYmd,AccumulateCovid19,areaNm,increase,AccumulateReleased,localIncrease,CovidForeigner
0,2022-01-01,6923.0,울산광역시,50.0,6371.0,50.0,0.0
4015,2022-01-01,8118.0,광주광역시,92.0,7349.0,91.0,1.0
2555,2022-01-01,226698.0,서울특별시,1464.0,177348.0,1429.0,35.0
3650,2022-01-01,15431.0,경상북도,116.0,13456.0,114.0,2.0
365,2022-01-01,2158.0,세종특별자치시,16.0,1957.0,16.0,0.0
...,...,...,...,...,...,...,...
5474,2022-12-31,1686864.0,인천광역시,3976.0,0.0,3975.0,1.0
5839,2022-12-31,947524.0,전라남도,2117.0,0.0,2116.0,1.0
364,2022-12-31,618200.0,울산광역시,1604.0,0.0,1603.0,1.0
2554,2022-12-31,1716966.0,부산광역시,4502.0,0.0,4501.0,1.0


In [6]:
# 미세먼지 데이터
pm = pd.read_csv('./data/pm.csv', parse_dates=['baseYmd'])
pm.head(3)

,areaNm,baseYmd,CAI,advisory,pmcategory
0,충청북도,2022-12-27,75,주의보,PM25
1,제주특별자치도,2022-12-14,213,주의보,PM10
2,경기도,2022-12-13,311,경보,PM10


In [7]:
# 공휴일 데이터
holiday = pd.read_csv('./data/holiday.csv', parse_dates=['baseYmd'])
holiday.head(3)

,holNm,baseYmd
0,1월1일,2022-01-01
1,설날,2022-01-31
2,설날,2022-02-01


In [8]:
# 관광객 집계 데이터와 날씨 데이터 합치기
result = pd.merge(df, weather, left_on=['baseYmd', 'areaNm'], right_on=['baseYmd', 'areaNm'], how='left')
result

,areaNm,daywkDivNm,touNum,baseYmd,avgTemp,dayRf,daySf
0,서울특별시,토요일,1171117.50,2022-01-01,-4.3,0.0,0.0
1,부산광역시,토요일,445102.00,2022-01-01,1.9,0.0,0.0
2,대구광역시,토요일,264113.50,2022-01-01,-2.3,0.0,0.0
3,인천광역시,토요일,496692.00,2022-01-01,-3.4,0.0,0.0
4,광주광역시,토요일,161028.00,2022-01-01,-0.1,0.0,0.0
...,...,...,...,...,...,...,...
6200,전라북도,NaN,221235.00,2022-09-03,25.3,0.0,0.0
6201,전라남도,NaN,304470.25,2022-09-03,24.1,0.5,0.0
6202,경상북도,NaN,425582.50,2022-09-03,21.0,0.1,0.0
6203,경상남도,NaN,363550.25,2022-09-03,21.0,6.6,0.0


In [9]:
# 강수량 연속형에서 범주형 변수로 전환
for index, Rf in result.iterrows():
    if Rf['dayRf'] == 0.0:
        result.at[index, 'dayRf'] = 'no'
    else:
        result.at[index, 'dayRf'] = 'yes'
        pass

In [10]:
# 강설량 연속형에서 범주형 변수로 전환
for index, Sf in result.iterrows():
    if Rf['daySf'] == 0.0:
        result.at[index, 'daySf'] = 'no'
    else:
        result.at[index, 'daySf'] = 'yes'
        pass

In [11]:
result.isnull().sum()

areaNm          0
daywkDivNm    119
touNum          0
baseYmd         0
avgTemp         0
dayRf           0
daySf           0
dtype: int64

In [12]:
result.loc[result['daywkDivNm'].isnull()]

,areaNm,daywkDivNm,touNum,baseYmd,avgTemp,dayRf,daySf
6086,서울특별시,NaN,1508164.00,2022-08-16,26.7,no,no
6087,부산광역시,NaN,353414.25,2022-08-16,26.1,yes,no
6088,대구광역시,NaN,249229.75,2022-08-16,25.7,yes,no
6089,인천광역시,NaN,399372.75,2022-08-16,25.0,no,no
6090,광주광역시,NaN,159759.25,2022-08-16,26.5,yes,no
...,...,...,...,...,...,...,...
6200,전라북도,NaN,221235.00,2022-09-03,25.3,no,no
6201,전라남도,NaN,304470.25,2022-09-03,24.1,yes,no
6202,경상북도,NaN,425582.50,2022-09-03,21.0,yes,no
6203,경상남도,NaN,363550.25,2022-09-03,21.0,yes,no


In [13]:
for index, day in result.iterrows():
    if day['baseYmd'] == pd.to_datetime('2022-08-16'):
        result.at[index, 'daywkDivNm'] = '화요일'
    elif day['baseYmd'] == pd.to_datetime('2022-08-17'):
        result.at[index, 'daywkDivNm'] = '수요일'
    elif day['baseYmd'] == pd.to_datetime('2022-08-18'):
        result.at[index, 'daywkDivNm'] = '목요일'
    elif day['baseYmd'] == pd.to_datetime('2022-08-19'):
        result.at[index, 'daywkDivNm'] = '금요일'
    elif day['baseYmd'] == pd.to_datetime('2022-08-20'):
        result.at[index, 'daywkDivNm'] = '토요일'
    elif day['baseYmd'] == pd.to_datetime('2022-09-02'):
        result.at[index, 'daywkDivNm'] = '금요일'
    elif day['baseYmd'] == pd.to_datetime('2022-09-03'):
        result.at[index, 'daywkDivNm'] = '토요일'
    else:
        pass

In [14]:
result.isnull().sum()

areaNm        0
daywkDivNm    0
touNum        0
baseYmd       0
avgTemp       0
dayRf         0
daySf         0
dtype: int64

In [15]:
# 축제여부 컬럼을 만들고 지역축제 데이터를 이용하여 축제여부 데이터 입력
for index, row in tqdm(result.iterrows()):
    for index2, row2 in festival.iterrows():
        if row['areaNm'] == row2['areaNm']:
            if (row2['fStart'] <= row['baseYmd']) and (row['baseYmd'] <= row2['fClose']):
                result.at[index, 'festival'] = 'yes'
            else:
                pass
        else:
            pass
# 결측값 0으로 채우기
result.fillna('no', inplace=True)


6205it [02:48, 36.87it/s]


In [16]:
result.isnull().sum()

areaNm        0
daywkDivNm    0
touNum        0
baseYmd       0
avgTemp       0
dayRf         0
daySf         0
festival      0
dtype: int64

In [17]:
# 공휴일 추가
result['holiday'] = 'no'

for index, row in tqdm(result.iterrows()):
    if row['daywkDivNm'] == '토요일' or row['daywkDivNm'] == '일요일':
        result.at[index, 'holiday'] = 'yes'
    for index2, row2 in holiday.iterrows():
        if row['baseYmd'] == row2['baseYmd']:
            result.at[index, 'holiday'] = 'yes'
        else:
            pass
result.drop(columns=['daywkDivNm'], inplace=True)

6205it [00:11, 540.03it/s]


In [18]:
# 적용 확인
# 2022-08-15 / 월요일이지만 광복절이므로 휴무일
result.loc[result['baseYmd'] == '2022-08-15']

,areaNm,touNum,baseYmd,avgTemp,dayRf,daySf,festival,holiday
3842,서울특별시,1320945.5,2022-08-15,28.1,yes,no,no,yes
3843,부산광역시,477795.5,2022-08-15,29.0,no,no,no,yes
3844,대구광역시,297668.0,2022-08-15,30.9,no,no,no,yes
3845,인천광역시,479258.5,2022-08-15,27.2,no,no,no,yes
3846,광주광역시,175934.5,2022-08-15,29.4,no,no,no,yes
3847,대전광역시,213062.5,2022-08-15,28.1,yes,no,no,yes
3848,울산광역시,158473.0,2022-08-15,30.2,no,no,yes,yes
3849,세종특별자치시,77794.5,2022-08-15,28.2,yes,no,no,yes
3850,경기도,1835466.5,2022-08-15,27.7,yes,no,no,yes
3851,강원도,742500.5,2022-08-15,27.5,yes,no,yes,yes


In [19]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6205 entries, 0 to 6204
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   areaNm    6205 non-null   object        
 1   touNum    6205 non-null   float64       
 2   baseYmd   6205 non-null   datetime64[ns]
 3   avgTemp   6205 non-null   float64       
 4   dayRf     6205 non-null   object        
 5   daySf     6205 non-null   object        
 6   festival  6205 non-null   object        
 7   holiday   6205 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 565.3+ KB


In [20]:
covid19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6205 entries, 0 to 6204
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   baseYmd             6205 non-null   datetime64[ns]
 1   AccumulateCovid19   6205 non-null   float64       
 2   areaNm              6205 non-null   object        
 3   increase            6205 non-null   float64       
 4   AccumulateReleased  6205 non-null   float64       
 5   localIncrease       6205 non-null   float64       
 6   CovidForeigner      6205 non-null   float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 339.5+ KB


In [21]:
# 미세먼지 데이터 추가
result['PM10'] = 'no'
result['PM25'] = 'no'

for row in tqdm(result.itertuples(), total=result.shape[0]):
    for row2 in pm.itertuples():
        if row2.pmcategory == 'PM10':
            if row.areaNm == row2.areaNm and row.baseYmd == row2.baseYmd:
                result.loc[row.Index, 'PM10'] = 'yes'
        elif row2.pmcategory == 'PM25':
            if row.areaNm == row2.areaNm and row.baseYmd == row2.baseYmd:
                result.loc[row.Index, 'PM25'] = 'yes'


100%|██████████| 6205/6205 [00:04<00:00, 1337.69it/s]


In [22]:
# 미세먼지 데이터 추가 확인
result.loc[(result['PM25'] == 'yes') | (result['PM10'] == 'yes')]

,areaNm,touNum,baseYmd,avgTemp,dayRf,daySf,festival,holiday,PM10,PM25
119,서울특별시,1551105.5,2022-01-08,0.3,no,no,no,yes,no,yes
122,인천광역시,434420.0,2022-01-08,1.6,no,no,no,yes,no,yes
127,경기도,1711720.5,2022-01-08,-2.4,no,no,no,yes,no,yes
141,대전광역시,196164.5,2022-01-09,2.5,no,no,no,yes,no,yes
143,세종특별자치시,62573.0,2022-01-09,1.8,no,no,no,yes,no,yes
...,...,...,...,...,...,...,...,...,...,...
5776,전라남도,233338.0,2022-12-13,6.9,no,no,no,no,yes,no
5777,경상북도,333417.0,2022-12-13,1.8,yes,no,no,no,yes,no
5778,경상남도,291889.5,2022-12-13,4.5,yes,no,no,no,yes,no
5796,제주특별자치도,145915.0,2022-12-14,4.2,yes,no,yes,no,yes,no


In [23]:
# 코로나 확진자 데이터 추가
result1 = pd.merge(result, covid19, left_on=['baseYmd', 'areaNm'], right_on=['baseYmd', 'areaNm'], how='left')

In [24]:
result1

,areaNm,touNum,baseYmd,avgTemp,dayRf,daySf,festival,holiday,PM10,PM25,AccumulateCovid19,increase,AccumulateReleased,localIncrease,CovidForeigner
0,서울특별시,1171117.50,2022-01-01,-4.3,no,no,no,yes,no,no,226698.0,1464.0,177348.0,1429.0,35.0
1,부산광역시,445102.00,2022-01-01,1.9,no,no,no,yes,no,no,25287.0,212.0,21163.0,210.0,2.0
2,대구광역시,264113.50,2022-01-01,-2.3,no,no,no,yes,no,no,23249.0,125.0,21394.0,125.0,0.0
3,인천광역시,496692.00,2022-01-01,-3.4,no,no,no,yes,no,no,36962.0,241.0,30703.0,239.0,2.0
4,광주광역시,161028.00,2022-01-01,-0.1,no,no,no,yes,no,no,8118.0,92.0,7349.0,91.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6200,전라북도,221235.00,2022-09-03,25.3,no,no,yes,yes,no,no,788103.0,3442.0,0.0,3431.0,11.0
6201,전라남도,304470.25,2022-09-03,24.1,yes,no,yes,yes,no,no,776823.0,3048.0,0.0,3041.0,7.0
6202,경상북도,425582.50,2022-09-03,21.0,yes,no,no,yes,no,no,1042687.0,4782.0,0.0,4753.0,29.0
6203,경상남도,363550.25,2022-09-03,21.0,yes,no,no,yes,no,no,1406209.0,5373.0,0.0,5362.0,11.0


In [25]:
print('서울특별시 : ',  len(result1.loc[result1['areaNm'] == '서울특별시']))
print('부산광역시 : ',  len(result1.loc[result1['areaNm'] == '부산광역시']))
print('대구광역시 : ',  len(result1.loc[result1['areaNm'] == '대구광역시']))
print('인천광역시 : ',  len(result1.loc[result1['areaNm'] == '인천광역시']))
print('광주광역시 : ',  len(result1.loc[result1['areaNm'] == '광주광역시']))
print('울산광역시 : ',  len(result1.loc[result1['areaNm'] == '울산광역시']))
print('강원도 : ',  len(result1.loc[result1['areaNm'] == '강원도']))
print('전라북도 : ',  len(result1.loc[result1['areaNm'] == '전라북도']))
print('전라남도 : ',  len(result1.loc[result1['areaNm'] == '전라남도']))
print('충청북도 : ',  len(result1.loc[result1['areaNm'] == '충청북도']))
print('충청남도 : ',  len(result1.loc[result1['areaNm'] == '충청남도']))
print('경상북도 : ',  len(result1.loc[result1['areaNm'] == '경상북도']))
print('경상남도 : ',  len(result1.loc[result1['areaNm'] == '경상남도']))
print('제주특별자치도 : ',  len(result1.loc[result1['areaNm'] == '제주특별자치도']))
print('세종특별자치시 : ',  len(result1.loc[result1['areaNm'] == '세종특별자치시']))
print('경기도 : ',  len(result1.loc[result1['areaNm'] == '경기도']))
print('대전광역시 : ',  len(result1.loc[result1['areaNm'] == '대전광역시']))

서울특별시 :  365
부산광역시 :  365
대구광역시 :  365
인천광역시 :  365
광주광역시 :  365
울산광역시 :  365
강원도 :  365
전라북도 :  365
전라남도 :  365
충청북도 :  365
충청남도 :  365
경상북도 :  365
경상남도 :  365
제주특별자치도 :  365
세종특별자치시 :  365
경기도 :  365
대전광역시 :  365


In [26]:
# 결측값 확인
result1.isnull().sum()

areaNm                0
touNum                0
baseYmd               0
avgTemp               0
dayRf                 0
daySf                 0
festival              0
holiday               0
PM10                  0
PM25                  0
AccumulateCovid19     0
increase              0
AccumulateReleased    0
localIncrease         0
CovidForeigner        0
dtype: int64

In [27]:
result1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6205 entries, 0 to 6204
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   areaNm              6205 non-null   object        
 1   touNum              6205 non-null   float64       
 2   baseYmd             6205 non-null   datetime64[ns]
 3   avgTemp             6205 non-null   float64       
 4   dayRf               6205 non-null   object        
 5   daySf               6205 non-null   object        
 6   festival            6205 non-null   object        
 7   holiday             6205 non-null   object        
 8   PM10                6205 non-null   object        
 9   PM25                6205 non-null   object        
 10  AccumulateCovid19   6205 non-null   float64       
 11  increase            6205 non-null   float64       
 12  AccumulateReleased  6205 non-null   float64       
 13  localIncrease       6205 non-null   float64     

In [28]:
# 중간 csv 저장
result1.to_csv('./data/result.csv', encoding='utf-8', index=False)

In [29]:
result1.drop(columns=['AccumulateReleased'], inplace=True)

In [30]:
# 최종 확인
result1.sort_values(['baseYmd'])

,areaNm,touNum,baseYmd,avgTemp,dayRf,daySf,festival,holiday,PM10,PM25,AccumulateCovid19,increase,localIncrease,CovidForeigner
0,서울특별시,1171117.5,2022-01-01,-4.3,no,no,no,yes,no,no,226698.0,1464.0,1429.0,35.0
16,제주특별자치도,176764.0,2022-01-01,4.9,no,no,no,yes,no,no,4661.0,23.0,22.0,1.0
15,경상남도,525623.0,2022-01-01,-2.2,no,no,no,yes,no,no,20840.0,162.0,162.0,0.0
14,경상북도,688541.0,2022-01-01,-7.1,no,no,no,yes,no,no,15431.0,116.0,114.0,2.0
13,전라남도,442116.5,2022-01-01,0.4,no,no,no,yes,no,no,6126.0,48.0,47.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6072,인천광역시,665108.5,2022-12-31,0.2,no,no,no,yes,no,no,1686864.0,3976.0,3975.0,1.0
6071,대구광역시,402769.5,2022-12-31,1.2,no,no,no,yes,no,no,1268852.0,2614.0,2610.0,4.0
6069,서울특별시,1847482.0,2022-12-31,-1.2,no,no,no,yes,no,no,5663251.0,11601.0,11597.0,4.0
6077,경기도,2251315.5,2022-12-31,-5.2,no,no,no,yes,no,no,7868406.0,16799.0,16794.0,5.0


In [31]:
# 전처리 최종 결과 csv 저장
result1.to_csv('./data/result_final.csv', encoding='utf-8', index=False)